In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from math import ceil


In [ ]:
## Storing all the files into dataframes

sample = pd.read_csv('../input/competitive-data-science-predict-future-sales/sample_submission.csv')
categories = pd.read_csv('../input/competitive-data-science-predict-future-sales/item_categories.csv')
items = pd.read_csv('../input/competitive-data-science-predict-future-sales/items.csv')
shops = pd.read_csv('../input/competitive-data-science-predict-future-sales/shops.csv')
sales_train = pd.read_csv('../input/competitive-data-science-predict-future-sales/sales_train.csv')
sales_test = pd.read_csv('../input/competitive-data-science-predict-future-sales/test.csv')

In [ ]:
sales_test

In [ ]:
sales_train.head()

In [ ]:
items.head()

In [ ]:
## Joining training data to the different item names
new_train = sales_train.join(items.set_index('item_id'),on='item_id')

In [ ]:
## Joining training data to the different item catagories
new_train = new_train.join(categories.set_index('item_category_id'),on='item_category_id')

In [ ]:
new_train = new_train.join(shops.set_index('shop_id'),on='shop_id')

In [ ]:
new_train['item_price'].nunique()

In [ ]:
new_train['item_name'].nunique()

In [ ]:
## Need to figure out how to extract total sales by date

## This gives us total sales by shop
new_train.groupby('shop_name').sum()

In [ ]:
new_train.groupby('date')

In [ ]:
## Looking to see how many rows we have
new_train.shape

In [ ]:
## Looking for outliers 
new_new = new_train[new_train['item_price']<=5000]

In [ ]:
## Looking at the shape of the item prices without 
new_new['item_price'].hist()

In [ ]:
outliers = new_train[new_train['item_price']>=5000]

In [ ]:
outliers

In [ ]:
33831/2935849

In [ ]:
train_prepped = new_train[new_train['item_price']<=5000]

In [ ]:
train_prepped

In [ ]:
## Attempts at seeing if it clusters

print(train_prepped['shop_id'].nunique())
print('\b')
print(train_prepped['date_block_num'].nunique())

print(60*34)

In [ ]:
train_prepped['date_time'] = pd.to_datetime(train_prepped['date'],format='%d.%m.%Y')
train_prepped

In [ ]:
df= train_prepped.groupby(['shop_id','date_block_num'],as_index=False)['item_cnt_day'].sum()
df


In [ ]:
df= train_prepped.groupby(['date_block_num','shop_id'],as_index=False)['item_cnt_day'].sum()
df


In [ ]:
## Graphing Monthly Sales number by company
plt.figure(figsize=[10,18])

sns.lineplot(data=df,x='date_block_num',y='item_cnt_day',hue='shop_id',palette='dark')
plt.legend(bbox_to_anchor=(1.2,1))

In [ ]:
monthly_sales=train_prepped.groupby(["date_block_num","shop_id","item_id"])[
    "date_time","item_price","item_cnt_day"].agg({"date_time":["min",'max'],"item_price":"mean","item_cnt_day":"sum"})

In [ ]:
df

In [ ]:
monthly_sales.head()

In [ ]:
grouped = pd.DataFrame(train_prepped.groupby(['shop_id', 'date_block_num'])['item_cnt_day'].sum().reset_index())
fig, axes = plt.subplots(nrows=5, ncols=2, sharex=True, sharey=True, figsize=(16,20))
num_graph = 10
id_per_graph = ceil(grouped.shop_id.max() / num_graph)
count = 0
for i in range(5):
    for j in range(2):
        sns.pointplot(x='date_block_num', y='item_cnt_day', hue='shop_id', data=grouped[np.logical_and(count*id_per_graph <= grouped['shop_id'], grouped['shop_id'] < (count+1)*id_per_graph)], ax=axes[i][j])
        count += 1

In [ ]:
len(train_prepped)

In [ ]:
len(sales_test)

In [ ]:
2902039/214200

In [ ]:
sales_test['item_id'].nunique()

In [ ]:
train_prepped['item_id'].nunique()

In [ ]:
## Creating minimum date for each item
min_date = train_prepped.groupby(["item_id"]).agg({"date_time":"min"})
min_date

In [ ]:

min_date['min_date'] = min_date['date_time']
min_date = min_date.drop('date_time',axis=1)

In [ ]:
## Joining date to minimum date & adding column to training data
train_prepped = train_prepped.join(min_date,on='item_id')

In [ ]:
train_prepped.head()

## Creating indicator for first month sale
def first_month(row):
    if (row['date_time'] > row['min_date']) & (row['date_time'] < (row['min_date'] + pd.Timedelta('30 d'))):
        return 1
    else:
        return 0
        
Incorrect way of going about it, gets the lowest date fore everything

In [ ]:
train_prepped['first_month'] = train_prepped.apply(first_month,axis=1)

In [ ]:
train_prepped.head()

In [ ]:
## train_prepped['first_month'] = (train_prepped['date_time'] > train_prepped['min_date']) & (train_prepped['date_time'] < (train_prepped['min_date'] + pd.Timedata('30 d'))

In [ ]:
## Creating minimum date for each item at each shop
min_date_per_shop = train_prepped.groupby(["shop_id","item_id"]).agg({"date_time":"min"})
min_date_per_shop

In [ ]:
## Creating indicator for first month sale at each shop
def first_month(row):
    if (row['date_time'] > min_date_per_shop.loc[(row['shop_id'],row['item_id']), 'date_time']) & (row['date_time'] < (min_date_per_shop.loc[(row['shop_id'],row['item_id']), 'date_time'] + pd.Timedelta('30 d'))):
        return 1
    else:
        return 0

In [ ]:
min_date_per_shop.loc[(row['shop_id'],row['item_id']), 'date_time']

In [ ]:
train_prepped['first_month'] = train_prepped.apply(first_month,axis=1)

In [ ]:
aggregated_train = train_prepped.groupby(["shop_id","item_id"])["date_time","item_cnt_day",'first_month'].agg({"date_time":["min",'max'],"item_cnt_day":"sum",'first_month':'sum'})
aggregated_train

In [ ]:
train_prepped.to_csv('training_prepped.csv')

In [ ]:
train_prepped.head()

In [ ]:
train_2 = train_prepped.groupby(['shop_id','item_id'])['item_cnt_day','first_month'].sum()

In [ ]:
sales_test.head()

In [ ]:
train_2.groupby('shop_id').mean()